Why should We Breakdown this
1. Understanding the problem is half of the solution
1. Understand the problem deeper
1. help to make strategic decision in preventing customer churn
1. estimate cost accurately and efficiently

# Data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df_telco = df_telco = pd.read_csv('Telco_customer_churn_clean.csv')

# Identify Main Problem

## 1. High Churn Rate (above 20%)

In [3]:
df_telco['Churn Label'].value_counts()

Churn Label
No     5174
Yes    1869
Name: count, dtype: int64

In [4]:
df_telco['Churn Label'].value_counts()/df_telco.shape[0]

Churn Label
No     0.73463
Yes    0.26537
Name: count, dtype: float64

PROBLEM 1 : 26.5% churn rate

## 2. Revenue Loss 30%

In [5]:
monet_value = 'Monthly Charges'
total_monthly_charges = df_telco[monet_value].sum()
monthly_charges_loss = sum(df_telco[monet_value] * df_telco['Churn Value'])
monthly_charges_keep = total_monthly_charges - monthly_charges_loss

In [6]:
charges = {
    'monthly charges all':total_monthly_charges,
    'monthly charges keep':monthly_charges_keep,
    'monthly charges keep (%)':round(100*monthly_charges_keep/total_monthly_charges,2),
    'monthly charges loss':monthly_charges_loss,
    'monthly charges loss (%)':round(100*monthly_charges_loss/total_monthly_charges,2),
}
charges

{'monthly charges all': 456116.6,
 'monthly charges keep': 316985.75,
 'monthly charges keep (%)': 69.5,
 'monthly charges loss': 139130.85,
 'monthly charges loss (%)': 30.5}

PROBLEM 2 : 30% revenue loss

## 3. Churn Reason

In [7]:
churn_reason_count = df_telco['Churn Reason'].value_counts().reset_index()
churn_reason_loss = df_telco[['Churn Reason','Monthly Charges']].groupby('Churn Reason').sum().sort_values('Monthly Charges', ascending = False).reset_index()
churn_reason_loss_pct = (df_telco[['Churn Reason','Monthly Charges']].groupby('Churn Reason').sum().sort_values('Monthly Charges', ascending = False)*100/df_telco[['Churn Reason','Monthly Charges']].groupby('Churn Reason').sum().sort_values('Monthly Charges', ascending = False).sum()).reset_index()
churn_reason_loss_pct = churn_reason_loss_pct.rename(columns = {'Monthly Charges':'Monthly Charges (%)'})
churn_reason_summary = pd.merge(pd.merge(churn_reason_count, churn_reason_loss),churn_reason_loss_pct)
churn_reason_summary

,Churn Reason,count,Monthly Charges,Monthly Charges (%)
0,Attitude of support person,192,13980.85,10.048706
1,Competitor offered higher download speeds,189,14144.60,10.166401
2,Competitor offered more data,162,12351.75,8.877794
3,Don't know,154,11099.05,7.977418
4,Competitor made better offer,140,10672.10,7.670549
5,Attitude of service provider,135,10399.30,7.474475
6,Competitor had better devices,130,9432.35,6.779481
7,Network reliability,103,7497.55,5.388848
8,Product dissatisfaction,102,7528.65,5.411201
9,Price too high,98,7398.55,5.317692


In [8]:
mapping_reason = {
    'Attitude of support person':'Support/Attitude Issue',
    'Attitude of service provider':'Support/Attitude Issue',
    'Poor expertise of phone support':'Support/Attitude Issue',
    'Poor expertise of online support':'Support/Attitude Issue',
    'Network reliability':'Stack Issue',
    'Product dissatisfaction':'Stack Issue',
    'Service dissatisfaction':'Stack Issue',
    'Limited range of services':'Stack Issue',
    'Lack of affordable download/upload speed':'Stack Issue',
    'Extra data charges':'Stack Issue',
    'Competitor offered more data':'Competitor',
    'Competitor made better offer':'Competitor',
    'Competitor had better devices':'Competitor',
    'Competitor offered higher download speeds':'Competitor',
    'Price too high':'Price',
    'Long distance charges':'Price',
    'Lack of self-service on Website':'Lack of self-service on Website',
    'Moved':'Moved',
    "Don't know":'Unknown',
    'Deceased':'Deceased'
}
churn_reason_summary['Churn Reason Mapped'] = churn_reason_summary['Churn Reason'].map(lambda x: mapping_reason.get(x, 'other')) 
churn_reason_summary_mapped = churn_reason_summary[['Churn Reason Mapped','count','Monthly Charges','Monthly Charges (%)']].groupby('Churn Reason Mapped').sum().reset_index().sort_values('count', ascending = False)

In [9]:
churn_reason_summary_mapped.to_excel('churn_reason_summary_mapped.xlsx')

# Breaking Down Main Problem into some Point Problem

## 1. User without additional internet service

### Problem Identification

In [10]:
def cat_add_on_service(security, backup, protection, support):
    if "Yes" in [security, backup, protection, support]:
        return 1
    else:
        return 0

df_telco['Additional Internet Service t'] = df_telco.apply(lambda x: cat_add_on_service(
    x['Online Security'],
    x['Online Backup'],
    x['Device Protection'],
    x['Tech Support']
), axis = 1)

In [11]:
df_telco_internet = df_telco[df_telco['Internet Service']!='No']

In [12]:
df_telco_internet['Additional Internet Service t'].value_counts()

Additional Internet Service t
1    4250
0    1267
Name: count, dtype: int64

In [13]:
df_telco_internet['Additional Internet Service t'].value_counts()/df_telco_internet.shape[0]

Additional Internet Service t
1    0.770346
0    0.229654
Name: count, dtype: float64

In [14]:
pd.crosstab(df_telco_internet['Additional Internet Service t'],df_telco_internet['Churn Label'], normalize = 0 )

Churn Label,No,Yes
Additional Internet Service t,,
0,0.433307,0.566693
1,0.755765,0.244235


PROBLEM 3 : half of user without additional internet service gone

### Further Analysis

In [15]:
pd.crosstab(df_telco_internet['Additional Internet Service t'],df_telco_internet['Churn Label'])

Churn Label,No,Yes
Additional Internet Service t,,
0,549,718
1,3212,1038


In [16]:
non_additional_inet_rev = sum((1-df_telco_internet['Additional Internet Service t'])*df_telco_internet['Churn Value']*df_telco_internet['Monthly Charges'])
non_additional_inet_rev_pct = 100*non_additional_inet_rev/total_monthly_charges

In [17]:
non_additional_inet_monetary = {
    'non additional inter rev':non_additional_inet_rev,
    'non additional inter rev(%)':non_additional_inet_rev_pct
}
non_additional_inet_monetary

{'non additional inter rev': 50806.05,
 'non additional inter rev(%)': 11.138829413356147}

## 2. New Customer

### Problem Identification

In [18]:
def tenure_year(tenure_months):
    if tenure_months < 12:
        return 0
    elif tenure_months < 24:
        return 1
    elif tenure_months < 36:
        return 2
    elif tenure_months < 48:
        return 3
    elif tenure_months < 60:
        return 4
    else:
        return 5

def newly_onboard(tenure_months):
    if tenure_months <= 3:
        return 1
    else:
        return 0

In [19]:
df_telco['Tenure Years'] = df_telco['Tenure Months'].apply(tenure_year)
df_telco['newly onboard'] = df_telco['Tenure Months'].apply(newly_onboard)

In [20]:
df_telco['Tenure Years'].value_counts()

Tenure Years
0    2069
5    1483
1    1047
2     876
4     820
3     748
Name: count, dtype: int64

In [21]:
pd.crosstab(df_telco['Tenure Years'], df_telco['Churn Label'], normalize = 0)

Churn Label,No,Yes
Tenure Years,,
0,0.517158,0.482842
1,0.704871,0.295129
2,0.779680,0.220320
3,0.804813,0.195187
4,0.850000,0.150000
5,0.933243,0.066757


In [22]:
df_telco['newly onboard'].value_counts()

newly onboard
0    5981
1    1062
Name: count, dtype: int64

In [23]:
pd.crosstab(df_telco['newly onboard'], df_telco['Churn Label'], normalize = 0)

Churn Label,No,Yes
newly onboard,,
0,0.787327,0.212673
1,0.437853,0.562147


PROBLEM 4 : half of new customer are tend to leave. It's also strongly related to contract type "Month"

### Further Analysis

In [24]:
pd.crosstab(df_telco['Tenure Years'], df_telco['Churn Label'])

Churn Label,No,Yes
Tenure Years,,
0,1070,999
1,738,309
2,683,193
3,602,146
4,697,123
5,1384,99


In [25]:
first_year_cust_rev = sum((df_telco['Tenure Years'] == 0)*df_telco['Churn Value']*df_telco['Monthly Charges'])
first_year_cust_rev_pct = 100*first_year_cust_rev/total_monthly_charges
first_year_cust_monetary = {
    'first year cust revenue':first_year_cust_rev,
    'first year cust revenue(%)':first_year_cust_rev_pct,
}
first_year_cust_monetary

{'first year cust revenue': 66022.3,
 'first year cust revenue(%)': 14.474873310903398}

In [26]:
pd.crosstab(df_telco['newly onboard'], df_telco['Churn Label'])

Churn Label,No,Yes
newly onboard,,
0,4709,1272
1,465,597


In [27]:
new_onboard_rev = sum((df_telco['newly onboard'] == 1)*df_telco['Churn Value']*df_telco['Monthly Charges'])
new_onboard_rev_pct = 100*new_onboard_rev/total_monthly_charges
new_onboard_rev_monetary = {
    'new_onboard_rev revenue':new_onboard_rev,
    'new_onboard_rev revenue(%)':new_onboard_rev_pct,
}
new_onboard_rev_monetary

{'new_onboard_rev revenue': 36428.7,
 'new_onboard_rev revenue(%)': 7.986707784807656}

## 3. People who use internet for streaming only, contribute to 6% revenue loss

## Problem Identification

In [28]:
def cat_add_on_streaming(internet, security, backup, protection, support, tv, movie):
    if internet != 'No' and 'Yes' not in [security, backup, protection, support] and 'Yes' in [tv, movie]:
        return 'streamer'
    else:
        return 'control'

In [29]:
df_telco['streamer'] = df_telco.apply(lambda x: cat_add_on_streaming(
    x['Internet Service'],
    x['Online Security'],
    x['Online Backup'],
    x['Device Protection'],
    x['Tech Support'],
    x['Streaming TV'],
    x['Streaming Movies']
), axis = 1)

In [30]:
df_telco_internet['streamer'] = df_telco_internet.apply(lambda x: cat_add_on_streaming(
    x['Internet Service'],
    x['Online Security'],
    x['Online Backup'],
    x['Device Protection'],
    x['Tech Support'],
    x['Streaming TV'],
    x['Streaming Movies']
), axis = 1)

C:\Users\muhyi\AppData\Local\Temp\ipykernel_7128\1857794611.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_telco_internet['streamer'] = df_telco_internet.apply(lambda x: cat_add_on_streaming(


In [31]:
df_telco_internet['streamer'].value_counts()

streamer
control     4943
streamer     574
Name: count, dtype: int64

In [32]:
pd.crosstab(df_telco_internet['streamer'], df_telco_internet['Churn Label'])

Churn Label,No,Yes
streamer,,
control,3543,1400
streamer,218,356


In [33]:
pd.crosstab(df_telco_internet['streamer'], df_telco_internet['Churn Label'], normalize = 0)

Churn Label,No,Yes
streamer,,
control,0.716771,0.283229
streamer,0.379791,0.620209


PROBLEM 5 : Most Streamer are gone

### Further Analysis

In [34]:
streamer_revenue = sum((df_telco_internet['streamer']=='streamer')*df_telco_internet['Monthly Charges'])
all_revenue = sum(df_telco['Monthly Charges'])
revenue_streamer_pct_all = 100*streamer_revenue/all_revenue

In [35]:
streamer_revenue_loss = sum((df_telco_internet['streamer']=='streamer')*df_telco_internet['Monthly Charges']*df_telco_internet['Churn Value'])
streamer_revenue_keep = streamer_revenue - streamer_revenue_loss

In [36]:
streamer_monetary = {
    'all revenue':all_revenue,
    'streamer revenue':streamer_revenue,
    'streamer revenue(%)':revenue_streamer_pct_all,
    'streamer revenue keep':streamer_revenue_keep,
    'streamer revenue keep (%)':100*streamer_revenue_keep/all_revenue,
    'streamer revenue loss':streamer_revenue_loss,
    'streamer revenue loss (%)':100*streamer_revenue_loss/all_revenue,
}
streamer_monetary

{'all revenue': 456116.6,
 'streamer revenue': 45270.05,
 'streamer revenue(%)': 9.92510467718123,
 'streamer revenue keep': 16679.250000000004,
 'streamer revenue keep (%)': 3.656795214206193,
 'streamer revenue loss': 28590.8,
 'streamer revenue loss (%)': 6.268309462975038}

## 4. Month-to-month Contract

### Problem identification

In [37]:
pd.crosstab(df_telco['Contract'], df_telco['Churn Label'])

Churn Label,No,Yes
Contract,,
Month-to-month,2220,1655
One year,1307,166
Two year,1647,48


In [38]:
pd.crosstab(df_telco['Contract'], df_telco['Churn Label'], normalize = 0)

Churn Label,No,Yes
Contract,,
Month-to-month,0.572903,0.427097
One year,0.887305,0.112695
Two year,0.971681,0.028319


### Further Analysis

In [39]:
contract_rev = sum((df_telco['Contract']=='Month-to-month')*df_telco['Churn Value']*df_telco['Monthly Charges'])
contract_rev_pct = 100*contract_rev/all_revenue

In [40]:
contract_monetary = {
    'contract rev':contract_rev,
    'contract rev (%)':contract_rev_pct
}
contract_monetary

{'contract rev': 120847.1, 'contract rev (%)': 26.494782255239123}

# Detected Problem Percentage

In [41]:
df_telco['p1'] = (1 - df_telco['Additional Internet Service t'])*df_telco['Churn Value']
df_telco['p2'] = (df_telco['Tenure Years']==0)*df_telco['Churn Value']
df_telco['p3'] = (df_telco['streamer']=='streamer')*df_telco['Churn Value']
df_telco['p4'] = (df_telco['Contract']=='Month-to-month')*df_telco['Churn Value']
df_telco['p5'] = (df_telco['Paperless Billing']=='Yes')*df_telco['Churn Value']
df_telco['p6'] = (df_telco['Payment Method']=='Electronic check')*df_telco['Churn Value']
df_telco['p7'] = (df_telco['Senior Citizen']=='Yes')*df_telco['Churn Value']

In [42]:
df_telco['problem'] = np.where(df_telco['p1'] | df_telco['p2'] | df_telco['p3'] | df_telco['p4'] | df_telco['p5'] | df_telco['p6']| df_telco['p7'],'identified','not identified')

In [43]:
df_telco[df_telco['Churn Label']=='Yes']['problem'].value_counts()

problem
identified        1840
not identified      29
Name: count, dtype: int64

In [44]:
df_telco[df_telco['Churn Label']=='Yes']['problem'].value_counts()/df_telco[df_telco['Churn Label']=='Yes']['problem'].shape[0]

problem
identified        0.984484
not identified    0.015516
Name: count, dtype: float64

In [45]:
loss_revenue_identified = sum((df_telco['problem'] == 'identified')*df_telco['Churn Value']*df_telco['Monthly Charges'])

In [46]:
loss_detected_monetary = {
    'loss revenue all':monthly_charges_loss,
    'loss revenue identified':loss_revenue_identified,
    'loss revenue identified pct':100*loss_revenue_identified/all_revenue,
}
loss_detected_monetary

{'loss revenue all': 139130.85,
 'loss revenue identified': 136642.0,
 'loss revenue identified pct': 29.957690643138182}

In [47]:
look_var = ['p1','p2','p3','p4','p5','p6','p7','p total']
df_telco_churn = df_telco[df_telco['Churn Value']==1]
df_telco_churn['p total'] = df_telco_churn[look_var[:-1]].sum(axis = 1)

C:\Users\muhyi\AppData\Local\Temp\ipykernel_7128\802457165.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_telco_churn['p total'] = df_telco_churn[look_var[:-1]].sum(axis = 1)


In [48]:
df_telco_churn['p total'].value_counts().sort_index()

p total
0     29
1    117
2    274
3    468
4    449
5    307
6    187
7     38
Name: count, dtype: int64

In [49]:
pd.crosstab(df_telco_churn['Churn Reason'],df_telco_churn['p1'], normalize = 1).sort_values(1, ascending = False)

p1,0,1
Churn Reason,,
Attitude of support person,0.093449,0.114320
Competitor offered higher download speeds,0.099229,0.103490
Don't know,0.072254,0.095066
Competitor made better offer,0.069364,0.081829
Attitude of service provider,0.072254,0.072202
Competitor offered more data,0.098266,0.072202
Competitor had better devices,0.073218,0.064982
Product dissatisfaction,0.053950,0.055355
Price too high,0.052023,0.052948


In [50]:
pd.crosstab(df_telco_churn['Churn Reason'],df_telco_churn['p2'], normalize = 1).sort_values(1, ascending = False)

p2,0,1
Churn Reason,,
Attitude of support person,0.105747,0.100100
Competitor offered higher download speeds,0.108046,0.095095
Competitor offered more data,0.087356,0.086086
Competitor made better offer,0.063218,0.085085
Don't know,0.085057,0.080080
Attitude of service provider,0.068966,0.075075
Competitor had better devices,0.063218,0.075075
Network reliability,0.043678,0.065065
Price too high,0.052874,0.052052


In [51]:
pd.crosstab(df_telco_churn['Churn Reason'],df_telco_churn['p3'], normalize = 1).sort_values(1, ascending = False)

p3,0,1
Churn Reason,,
Competitor offered higher download speeds,0.099141,0.109551
Attitude of support person,0.102445,0.103933
Don't know,0.080635,0.089888
Competitor made better offer,0.073364,0.081461
Attitude of service provider,0.072042,0.073034
Competitor offered more data,0.090549,0.070225
Competitor had better devices,0.070059,0.067416
Network reliability,0.053536,0.061798
Product dissatisfaction,0.054858,0.053371


In [52]:
pd.crosstab(df_telco_churn['Churn Reason'],df_telco_churn['p4'], normalize = 1).sort_values(1, ascending = False)

p4,0,1
Churn Reason,,
Attitude of support person,0.084112,0.105136
Competitor offered higher download speeds,0.130841,0.097281
Don't know,0.060748,0.085196
Competitor offered more data,0.102804,0.084592
Competitor made better offer,0.065421,0.076133
Competitor had better devices,0.065421,0.070091
Attitude of service provider,0.093458,0.069486
Network reliability,0.051402,0.055589
Price too high,0.042056,0.053776


In [53]:
pd.crosstab(df_telco_churn['Churn Reason'],df_telco_churn['p5'], normalize = 1).sort_values(1, ascending = False)

p5,0,1
Churn Reason,,
Competitor offered higher download speeds,0.095949,0.102857
Attitude of support person,0.106610,0.101429
Competitor offered more data,0.081023,0.088571
Don't know,0.085288,0.081429
Competitor made better offer,0.074627,0.075000
Attitude of service provider,0.068230,0.073571
Competitor had better devices,0.074627,0.067857
Network reliability,0.059701,0.053571
Product dissatisfaction,0.059701,0.052857


In [54]:
pd.crosstab(df_telco_churn['Churn Reason'],df_telco_churn['p6'], normalize = 1).sort_values(1, ascending = False)

p6,0,1
Churn Reason,,
Competitor offered higher download speeds,0.096491,0.104575
Attitude of support person,0.102757,0.102708
Competitor offered more data,0.086466,0.086835
Attitude of service provider,0.065163,0.077498
Competitor made better offer,0.071429,0.077498
Don't know,0.091479,0.075630
Competitor had better devices,0.066416,0.071895
Network reliability,0.051378,0.057890
Product dissatisfaction,0.052632,0.056022


In [55]:
pd.crosstab(df_telco_churn['Churn Reason'],df_telco_churn['p7'], normalize = 1).sort_values(1, ascending = False)

p7,0,1
Churn Reason,,
Competitor offered higher download speeds,0.100503,0.102941
Competitor made better offer,0.066045,0.100840
Attitude of support person,0.107681,0.088235
Attitude of service provider,0.069634,0.079832
Competitor offered more data,0.089734,0.077731
Price too high,0.045944,0.071429
Competitor had better devices,0.071070,0.065126
Don't know,0.088299,0.065126
Product dissatisfaction,0.053123,0.058824


# Summary

In [56]:
churn_rate_p1 = df_telco[(df_telco['Additional Internet Service t'] == 0)&(df_telco['Internet Service'] != 'No')]['Churn Value'].mean()
churn_rate_p2 = df_telco[(df_telco['Tenure Years'] == 0)]['Churn Value'].mean()
churn_rate_p3 = df_telco[df_telco['streamer']=='streamer']['Churn Value'].mean()
churn_rate_p4 = df_telco[df_telco['Contract']=='Month-to-month']['Churn Value'].mean()
churn_rate_p5 = df_telco[df_telco['Paperless Billing']=='Yes']['Churn Value'].mean()
churn_rate_p6 = df_telco[df_telco['Payment Method']=='Electronic check']['Churn Value'].mean()
churn_rate_p7 = df_telco[df_telco['Senior Citizen']=='Yes']['Churn Value'].mean()

In [57]:
loss_p1 = (df_telco[(df_telco['Additional Internet Service t'] == 0)&(df_telco['Internet Service'] != 'No')]['Churn Value']*df_telco['Monthly Charges']).sum()
loss_p2 = (df_telco[(df_telco['Tenure Years'] == 0)]['Churn Value']*df_telco['Monthly Charges']).sum()
loss_p3 = (df_telco[df_telco['streamer']=='streamer']['Churn Value']*df_telco['Monthly Charges']).sum()
loss_p4 = (df_telco[df_telco['Contract']=='Month-to-month']['Churn Value']*df_telco['Monthly Charges']).sum()
loss_p5 = (df_telco[df_telco['Paperless Billing']=='Yes']['Churn Value']*df_telco['Monthly Charges']).sum()
loss_p6 = (df_telco[df_telco['Payment Method']=='Electronic check']['Churn Value']*df_telco['Monthly Charges']).sum()
loss_p7 = (df_telco[df_telco['Senior Citizen']=='Yes']['Churn Value']*df_telco['Monthly Charges']).sum()

In [58]:
churn_rate_breakdown = [churn_rate_p1, churn_rate_p2, churn_rate_p3, churn_rate_p4, churn_rate_p5, churn_rate_p6, churn_rate_p7]
loss_breakdown = [loss_p1, loss_p2, loss_p3, loss_p4, loss_p5, loss_p6, loss_p7]
segments = [
    'p1 : Internet user without additional services', 
    'p2 : New Customer, oboarded less than a year',
    'p3 : Movie Streamer and TV Streamer',
    'p4 : User with month-to-month contract',
    'p5 : Paperless Billing',
    'p6 : Electronic check',
    'p7 : Senior Citizen'
]
churn_rate_per_segment = pd.DataFrame({
    'segment':segments,
    'churn rate':churn_rate_breakdown,
    'loss':loss_breakdown
})
churn_rate_per_segment

,segment,churn rate,loss
0,p1 : Internet user without additional services,0.566693,50806.05
1,"p2 : New Customer, oboarded less than a year",0.482842,66022.30
2,p3 : Movie Streamer and TV Streamer,0.620209,28590.80
3,p4 : User with month-to-month contract,0.427097,120847.10
4,p5 : Paperless Billing,0.335651,109510.00
5,p6 : Electronic check,0.452854,84288.75
6,p7 : Senior Citizen,0.416813,38419.60


In [59]:
churn_rate_per_segment.to_excel('churn rate breakdown.xlsx', index = False)

# Save Data

In [60]:
df_telco.to_csv('Telco_customer_churn_clean.csv', index = False)